In [ ]:
import cirq
import numpy as np

# Function to create an oracle gate from a given unitary matrix
def create_oracle_gate(oracle_matrix):
    return cirq.MatrixGate(oracle_matrix)

# Function to prompt the user to input a 4x4 unitary matrix
def input_unitary_matrix():
    try:
        n = int(input("Enter n (number of input qubits): ").strip() or "1")
    except ValueError:
        n = 1

    print(f"\nWe will define a function f(x) that takes an n-bit input x and returns a single bit (0 or 1).")
    print(f"When you see something like f(0, 1), it means x = (0, 1) (the input qubit values).")
    print(f"Please enter the output bit f(x) for each input x.\n")

    f_values = {}
    for x in range(2**n):
        bits = format(x, f"0{n}b")
        label = ", ".join(bits)
        while True:
            raw = input(f"For input x = ({label}), enter the output f(x) (0 or 1): ").strip()
            if raw in {"0", "1"}:
                f_values[x] = int(raw)
                break
            print("Please enter 0 or 1.")

    dim = 2 ** (n + 1)
    oracle_matrix = np.zeros((dim, dim), dtype=float)

    for x in range(2**n):
        fx = f_values[x]
        for y in (0, 1):
            in_index = (x << 1) | y
            out_y = y ^ fx
            out_index = (x << 1) | out_y
            oracle_matrix[out_index, in_index] = 1.0

    return oracle_matrix, n

# Define Deutsch's algorithm
def deutsch_algorithm(oracle_gate, n):
    # Define qubits
    qubits = cirq.LineQubit.range(n + 1)
    x_qubits = qubits[:n]
    y_qubit = qubits[n]

    # Create a quantum circuit
    circuit = cirq.Circuit()

    # Apply Hadamard gate to both qubits to create superposition
    circuit.append(cirq.X(y_qubit))
    circuit.append(cirq.H.on_each(*qubits))

    # Apply the oracle gate
    circuit.append(oracle_gate(*qubits))

    # Apply Hadamard gate to the first qubit only
    circuit.append(cirq.H.on_each(*x_qubits))

    # Measure the first qubit
    circuit.append(cirq.measure(*x_qubits, key='result'))

    return circuit

# Prompt the user to input the 4x4 unitary matrix
oracle_matrix, n = input_unitary_matrix()

# Create the oracle gate from the user-defined unitary matrix
oracle_gate = create_oracle_gate(oracle_matrix)

# Run Deutsch's algorithm with the user-defined oracle gate
circuit = deutsch_algorithm(oracle_gate, n)

simulator = cirq.Simulator()
measurement_outcomes = []

for _ in range(10):
    result = simulator.run(circuit)
    measurement_outcome = result.measurements['result'][0]
    measurement_outcomes.append(measurement_outcome)

# Print all measurement outcomes
print("Measurement outcomes:", measurement_outcomes)

# Determine the type of function based on the measurement outcomes
function_type = "constant" if all(np.all(outcome == 0) for outcome in measurement_outcomes) else "balanced"
print("Function type:", function_type)

# Print the circuit
print("\nCircuit:")
print(circuit)


Measurement outcomes: [array([1, 0], dtype=int8), array([1, 0], dtype=int8), array([1, 0], dtype=int8), array([1, 0], dtype=int8), array([1, 0], dtype=int8), array([1, 0], dtype=int8), array([1, 0], dtype=int8), array([1, 0], dtype=int8), array([1, 0], dtype=int8), array([1, 0], dtype=int8)]
Function type: balanced

Circuit:
              ┌                       ┐
              │0. 1. 0. 0. 0. 0. 0. 0.│
              │1. 0. 0. 0. 0. 0. 0. 0.│
              │0. 0. 0. 1. 0. 0. 0. 0.│
0: ───H───────│0. 0. 1. 0. 0. 0. 0. 0.│───H───M('result')───
              │0. 0. 0. 0. 1. 0. 0. 0.│       │
              │0. 0. 0. 0. 0. 1. 0. 0.│       │
              │0. 0. 0. 0. 0. 0. 1. 0.│       │
              │0. 0. 0. 0. 0. 0. 0. 1.│       │
              └                       ┘       │
              │                               │
1: ───H───────#2──────────────────────────H───M─────────────
              │
2: ───X───H───#3────────────────────────────────────────────
